In [ ]:
import xarray as xr
import glob

# Path to data
path = '/Volumes/external/TIW/data/glorys/'

# Find files
files = glob.glob(f'{path}*.nc')

# Separate files by resolution
high_res_files = [f for f in files if '0.083deg' in f]
low_res_files = [f for f in files if '0.25deg' in f]

# Variables we want to keep
vars_to_keep = ['thetao', 'so', 'uo', 'vo']

# Open high-resolution data lazily and drop unnecessary variables
ds_high_res = xr.open_mfdataset(
    high_res_files,
    combine='by_coords',
    chunks={'time': 100},  # Adjust for your machine
    parallel=True,
    drop_variables=[var for var in xr.open_dataset(high_res_files[0]).data_vars if var not in vars_to_keep]
)

# Load low-resolution file
ds_low_res = xr.open_dataset(low_res_files[0])

print('files are opened')

# Rename low-resolution variables to match high-resolution
rename_dict = {
    'thetao_glor': 'thetao',
    'so_glor': 'so',
    'uo_glor': 'uo',
    'vo_glor': 'vo'
}
ds_low_res = ds_low_res.rename(rename_dict)

# Select only needed variables after renaming
ds_low_res = ds_low_res[vars_to_keep]

# Align low-res (0.25°) to high-res (0.083°) grid if necessary
if ds_low_res.longitude.shape != ds_high_res.longitude.shape or ds_low_res.latitude.shape != ds_high_res.latitude.shape:
    ds_low_res = ds_low_res.interp(
        longitude=ds_high_res.longitude,
        latitude=ds_high_res.latitude,
        method='linear'
    )


ds_high_res_before = ds_high_res.sel(time=slice(None, "2021-06-29"))
ds_low_res_gap = ds_low_res.sel(time=slice("2021-06-30", "2022-06-01"))
ds_high_res_after = ds_high_res.sel(time=slice("2022-06-02", None))


combined = xr.concat(
    [ds_high_res_before, ds_low_res_gap, ds_high_res_after],
    dim='time',
    #combine_attrs='override'
)
combined = combined.sortby("time")

# Save final combined dataset
combined.to_netcdf(f'{path}/glorys_2017_2025_cleaned_0.083deg.nc', engine='netcdf4', compute=True)

# issue: not properly concactonating the low resoltuion data to the high resolution data gap...

attempt to clean the data... maybe just do it aug-feb of each year and save as netcdf chunks.

In [ ]:
import xarray as xr
import numpy as np
from scipy.signal import butter, filtfilt, hilbert
import os
import glob

# Paths
path = '/Volumes/external/TIW/data/glorys/'
output_dir = os.path.join(path, 'winters')
phase_dir = '/Users/katiekohlman/Desktop/TIW/netCDF/'
os.makedirs(output_dir, exist_ok=True)
os.makedirs(phase_dir, exist_ok=True)

# Find files
files = glob.glob(f'{path}*.nc')

# Separate files by resolution
high_res_files = [f for f in files if '0.083deg' in f]
low_res_files = [f for f in files if '0.25deg' in f]

# Variables to process
vars_to_keep = ['thetao', 'so', 'uo', 'vo']

# Target chunking (adjust based on your machine)
target_chunks = {'time': 10, 'depth': 50, 'latitude': 200, 'longitude': 200}

# Open high-res data lazily
ds_high_res = xr.open_mfdataset(
    high_res_files,
    combine='by_coords',
    chunks=target_chunks,
    parallel=True,
    drop_variables=[var for var in xr.open_dataset(high_res_files[0]).data_vars if var not in vars_to_keep]
)

# Open and rename low-res data
ds_low_res = xr.open_dataset(low_res_files[0], chunks=target_chunks)
ds_low_res = ds_low_res.rename({
    'thetao_glor': 'thetao',
    'so_glor': 'so',
    'uo_glor': 'uo',
    'vo_glor': 'vo'
})[vars_to_keep]

# Interpolate low-res to high-res grid (one-time process if not already saved)
interp_path = f'{path}/low_res_interp_to_high_res_grid.zarr'

if not os.path.exists(interp_path):
    ds_low_res = ds_low_res.interp(
        longitude=ds_high_res.longitude,
        latitude=ds_high_res.latitude,
        method='linear'
    )
    ds_low_res.to_zarr(interp_path, mode='w')
else:
    ds_low_res = xr.open_zarr(interp_path)

# Time slices (gap handling)
ds_high_res_before = ds_high_res.sel(time=slice(None, "2021-06-29"))
ds_low_res_gap = ds_low_res.sel(time=slice("2021-06-30", "2022-06-01"))
ds_high_res_after = ds_high_res.sel(time=slice("2022-06-02", None))

print("interpolating done")

# Define winter season ranges
seasons = [
    ('2017-08-01', '2018-02-28'),
    ('2018-08-01', '2019-02-28'),
    ('2019-08-01', '2020-02-29'),  # Leap year!
    ('2020-08-01', '2021-02-28'),
    ('2021-08-01', '2022-02-28'),
    ('2022-08-01', '2023-02-28'),
    ('2023-08-01', '2024-02-29'),
    ('2024-08-01', '2025-02-28'),
]

# Function to compute phase signal for temperature
def compute_phase_signal(winter_file, start, end):
    ds = xr.open_dataset(winter_file)

    # Surface temperature processing
    sfc_depth = 0
    temp_sfc = ds.thetao.sel(depth=sfc_depth, method='nearest')

    # Time mean and anomaly
    temp_time_mean_sfc = temp_sfc.mean(dim="time")
    temp_anomaly_sfc = temp_sfc - temp_time_mean_sfc

    # Detrend
    trend_sfc = temp_anomaly_sfc.polyfit(dim="time", deg=1)
    trend_fit_sfc = xr.polyval(temp_sfc.time, trend_sfc.polyfit_coefficients)
    temp_anomaly_detrended_sfc = temp_anomaly_sfc - trend_fit_sfc

    # Bandpass filter
    time_step_days = np.mean(np.diff(temp_sfc.time).astype('timedelta64[s]').astype(int)) / (60 * 60 * 24)
    nyquist = 0.5 * (1 / time_step_days)
    low_cutoff = 1 / 50
    high_cutoff = 1 / 15
    low, high = low_cutoff / nyquist, high_cutoff / nyquist

    b, a = butter(4, [low, high], btype='band')
    filtered_temp_sfc = filtfilt(b, a, temp_sfc)

    # Create dataset for phase processing
    ds_phase = xr.Dataset(
        {
            "temp_sfc": (["time", "latitude", "longitude"], temp_sfc.data),
            "temp_anomaly_sfc": (["time", "latitude", "longitude"], temp_anomaly_sfc.data),
            "temp_butter_sfc": (["time", "latitude", "longitude"], filtered_temp_sfc),
            "temp_anomaly_detrended_sfc": (["time", "latitude", "longitude"], temp_anomaly_detrended_sfc.data),
        },
        coords={
            "time": temp_sfc.time,
            "latitude": ds.latitude,
            "longitude": ds.longitude,
        }
    )

    # Hilbert transform and phase computation
    for var_name in ['temp_butter_sfc', 'temp_anomaly_sfc', 'temp_anomaly_detrended_sfc']:
        analytic_signal = hilbert(ds_phase[var_name], axis=0)
        phase = np.angle(analytic_signal, deg=False)
        phase_var_name = var_name.replace('temp_', '').replace('_sfc', '')
        ds_phase[f'phase_{phase_var_name}'] = (('time', 'latitude', 'longitude'), phase)

    # Save phase file
    phase_file = f'{phase_dir}/glorys_thetao_phase_{start[:4]}_{end[:4]}.nc'
    ds_phase.to_netcdf(phase_file)
    print(f'Saved phase data to: {phase_file}')

# Process each variable and season
for var in vars_to_keep:
    for start, end in seasons:
        if start <= "2021-06-29":
            ds_season = ds_high_res_before[var].sel(time=slice(start, end))
        elif start >= "2022-06-02":
            ds_season = ds_high_res_after[var].sel(time=slice(start, end))
        else:
            ds_part1 = ds_high_res_before[var].sel(time=slice(start, "2021-06-29"))
            ds_part2 = ds_low_res_gap[var].sel(time=slice("2021-06-30", "2022-06-01"))
            ds_part3 = ds_high_res_after[var].sel(time=slice("2022-06-02", end))
            ds_season = xr.concat([ds_part1, ds_part2, ds_part3], dim='time').sortby('time')

        if len(ds_season.time) == 0:
            continue

        winter_file = os.path.join(output_dir, f'glorys_{var}_winter_{start[:4]}_{end[:4]}.nc')
        ds_season.to_netcdf(winter_file)

        # Run phase computation for temperature after saving thetao
        if var == 'thetao':
            compute_phase_signal(winter_file, start, end)

print("Processing complete.")


In [ ]:
print(ds_high_res_before.time.values)
print(ds_low_res_gap.time.values)
print(ds_high_res_after.time.values)

print(ds_high_res.latitude.values)
print(ds_low_res_gap.latitude.values)

print(ds_high_res.longitude.values)
print(ds_low_res_gap.longitude.values)


In [ ]:
combined = xr.combine_by_coords([ds_high_res_before, ds_low_res_gap, ds_high_res_after], combine_attrs='override')


In [ ]:
plt.plot(ds_high_res_before.time,ds_high_res_before.sel(depth = 0, latitude = 2, longitude =-140, method='nearest').thetao)
plt.plot(ds_low_res_gap.time,ds_low_res_gap.sel(depth = 0, latitude = 2, longitude =-140, method='nearest').thetao)
plt.plot(ds_high_res_after.time,ds_high_res_after.sel(depth = 0, latitude = 2, longitude =-140, method='nearest').thetao)
plt.xlim(np.datetime64("2021-01-30"), np.datetime64("2023-01-30"))


plt.plot(combined2.time,combined2.sel(depth = 0, latitude = 2, longitude =-140, method='nearest').thetao)